#  <b><span style="color:#14c8b3"><center> Preprocessing Steps  </span></b> </center>


   

<span> <center><img src="https://miro.medium.com/max/1200/1*sjDV85QYa0quGsjRJtqN1g.jpeg" width="700" /></span>

In [1]:
#Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import Word

import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
import httpimport
url = 'https://raw.githubusercontent.com/zach-brown-18/class-toolkit/main/eda/'
with httpimport.remote_repo(['cleaning', 'columns'], url):
    import cleaning as c
    import columns as cols

In [3]:
df=pd.read_excel('Linkedin Job Posts in Saudi Arabia 2020.xlsx')

In [4]:
df.head()

,linkedin_id,position_id,position,company,location,level,date,job_functions,industries,description
0,1632901070,51791,Head of Branding Governance Unit,Riyad Bank,"Riyadh, Saudi Arabia",Mid-Senior level,2020-01-06,"['Marketing', 'Management']",['Banking'],NaN
1,1628497696,51792,Document Controller,"Confidential,Confidential","Riyadh, Saudi Arabia",Associate,2020-01-06,[''],['Government Administration'],Document Controller- (Saudi candidates only) ...
2,1666951758,51793,Employee Engagement Specialist,PepsiCo,"Riyadh, Saudi Arabia",Mid-Senior level,2020-01-06,['Human Resources'],"['Consumer Goods', 'Food & Beverages']",Auto req ID: 199735BRJob DescriptionPepsiCo Sa...
3,1671138260,51794,Strategic Workforce Analyst,Saudi Air Navigation Services,"Jeddah, Saudi Arabia",Mid-Senior level,2020-01-06,[''],['Aviation & Aerospace'],Role PurposeTo conduct workforce analysis to s...
4,1670725370,51795,"Account Manager, GPS – Riyadh",EY,"Riyadh, Saudi Arabia",Not Applicable,2020-01-06,"['Sales', 'Business Development']","['Accounting', 'Financial Services']",In a business where are our people are our pro...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48449 entries, 0 to 48448
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   linkedin_id    48443 non-null  object        
 1   position_id    48449 non-null  int64         
 2   position       48449 non-null  object        
 3   company        48447 non-null  object        
 4   location       48436 non-null  object        
 5   level          46758 non-null  object        
 6   date           48449 non-null  datetime64[ns]
 7   job_functions  48449 non-null  object        
 8   industries     48449 non-null  object        
 9   description    45252 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 3.7+ MB


In [6]:
#Print Null Values
df.isnull().sum()

linkedin_id         6
position_id         0
position            0
company             2
location           13
level            1691
date                0
job_functions       0
industries          0
description      3197
dtype: int64

#### Define variable contain the special character to cleaning purpose

In [7]:
#define special character to rmove it from data set
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

# Preprocessing and Cleaning of Data

#### Extract two columns from date column

In [8]:
#extract day from date
df['day'] = df['date'].dt.day

In [9]:
#extract month from date
df['month'] = df['date'].dt.month

In [10]:
#extract quarter from date
df['quarter'] = df['date'].dt.quarter

### Location column Cleaning and extacting New column city from it 

In [11]:
for char in spec_chars:
    df['location'] = df['location'].str.replace(char, ' ')

#### delete space for good cleaning

In [12]:
#delete space 
df['location'] = df['location'].str.replace('Al ','Al')
df['location'] = df['location'].str.replace('Ad ','Ad')
df['location'] = df['location'].str.replace('Om Ashr ','OmAshr')
df['location'] = df['location'].str.replace('Ha il ','Hail')
df['location'] = df['location'].str.replace('Le Méridien ','LeMéridien')
df['location'] = df['location'].str.replace('Al‘Ulá ','AlUla')
df['location'] = df['location'].str.replace('As sa an  Syria ','Riyadh')
df['location'] = df['location'].str.replace('Ar Rass ','AlQasim')
df['location'] = df['location'].str.replace('Aţ Ţā’if ','Taif')
df['location'] = df['location'].str.replace('As Sulimaniyah ','Riyadh')
df['location'] = df['location'].str.replace('Abu Shuayb ','Makkah')
df['location'] = df['location'].str.replace('AlImam Muhammad ','Riyadh')
df['location'] = df['location'].str.replace('Ta if ','Taif')
df['location'] = df['location'].str.replace('UmmalHamam','UmmalHamam ')
df['location'] = df['location'].str.replace('At Turqi ','Makkah')
df['location'] = df['location'].str.replace("‘Ar‘ar ",'Arar')
df['location'] = df['location'].str.replace('Abu Urwah ','Makkah')






In [13]:
print(df['location'].unique())

['Riyadh  Saudi Arabia' 'Jeddah  Saudi Arabia' 'Saudi Arabia'
 'Dammam  Saudi Arabia' 'Khobar  Saudi Arabia' 'Rabigh  Saudi Arabia'
 'Makkah  Saudi Arabia' nan 'Yanbu  Saudi Arabia' 'Dhahran  Saudi Arabia'
 'Madinah  Saudi Arabia' 'Jubail  Saudi Arabia' 'Jizan  Saudi Arabia'
 'Ahsa  Saudi Arabia' 'Tabouk  Saudi Arabia' 'Qaseem  Saudi Arabia'
 'Taif  Saudi Arabia' 'Najran  Saudi Arabia' 'Hail  Saudi Arabia'
 'Baha  Saudi Arabia' 'Jeddah  Makkah  Saudi Arabia'
 'Dammam Industrial Area  Eastern  Saudi Arabia' 'Tabuk  Saudi Arabia'
 'Ḩafar al Baţin  Saudi Arabia' 'Eastern  Saudi Arabia'
 'AdDammām  Eastern  Saudi Arabia' 'Bedaah  AlMadinah  Saudi Arabia'
 'AlKharj  Riyadh  Saudi Arabia' 'AlKhubar  Eastern  Saudi Arabia'
 'AlKhobar  Eastern  Saudi Arabia'
 'AlJubail Industrial  Eastern  Saudi Arabia'
 'Jeddah Industrial City 1  Makkah  Saudi Arabia'
 'Dammam Second Industrial City  Eastern  Saudi Arabia'
 'Medina  AlMadinah  Saudi Arabia' 'Qal‘at Bīshah   Asir  Saudi Arabia'
 'AlJubayl  Sau

In [14]:
#extract first word from location column
df['city'] = df['location'].str.split(' ').str[0]

In [15]:
print(df['city'].unique())

['Riyadh' 'Jeddah' 'Saudi' 'Dammam' 'Khobar' 'Rabigh' 'Makkah' nan 'Yanbu'
 'Dhahran' 'Madinah' 'Jubail' 'Jizan' 'Ahsa' 'Tabouk' 'Qaseem' 'Taif'
 'Najran' 'Hail' 'Baha' 'Tabuk' 'Ḩafar' 'Eastern' 'AdDammām' 'Bedaah'
 'AlKharj' 'AlKhubar' 'AlKhobar' 'AlJubail' 'Medina' 'Qal‘at' 'AlJubayl'
 'AdDawādimī' 'AlMajal' 'Mecca' '' 'AlYasmeen' 'AlMadinah' 'Beesh'
 'Ḩuraymila' 'AlQasim' 'AlBakhraa' 'AlHufūf' 'First' 'AlMuthneb' 'Khamīs'
 'AlJazirah' 'Dasimah' 'Rābigh' 'Abhā' 'Chalon' 'AlMalga' 'AlMalaz'
 'Tuwaiq' 'AlDhahran' 'AlSulaimaniyah' 'Sakākah' 'Jawharat' 'AlBadai'
 'AlGhamas' 'AlHamra' 'AlUlayah' 'AlWaha' 'AlRoudah' 'Azzarqa' 'AlRaid'
 'AlRahmaniyah' 'AlWorood' 'AlBahah' 'AlNuzha' 'Buraydah' 'AlQadesiyah'
 'Yadma' 'Ţurayf' 'Badr' 'Qurayyat' 'AlBukayrīyah' 'Sulaimaniyah'
 'Ishbayliyah' 'Yanbu‘' 'AlAflaj' 'Radhwa' 'Hinakiyah' 'King' 'Alsnaeia'
 'Alsharqiyah' 'Alehsaa' 'Ḑubā' 'Ashifa' '‘Unayzah' 'Ḩā’il' 'Jīzān'
 'AlMajma‘ah' 'AlKhafji' 'AlLīth' 'Ras' 'AlShuqaiq' 'AlSaadah' 'Quwayiyah'
 'Ain' 

In [16]:
df['city'] = df['city'].str.replace('AdDammām','Dammam')
df['city'] = df['city'].str.replace('Jiddah','Jeddah')
df['city'] = df['city'].str.replace('Mecca','Makkah')
df['city'] = df['city'].str.replace('J카z훮n','Jazan')
df['city'] = df['city'].str.replace('Jizan','Jazan')
df['city'] = df['city'].str.replace('Jīzān','Jazan')
df['city'] = df['city'].str.replace('Tabouk','Tabuk')
df['city'] = df['city'].str.replace('Tabūk','Tabuk')
df['city'] = df['city'].str.replace('Medina','Madinah')
df['city'] = df['city'].str.replace('AlMadinah','Madinah')
df['city'] = df['city'].str.replace('R훮bigh','Rabigh')
df['city'] = df['city'].str.replace('Rābigh','Rabigh')
df['city'] = df['city'].str.replace('AlJubayl','Jubail')
df['city'] = df['city'].str.replace('AlJubail','Jubail')
df['city'] = df['city'].str.replace('AlBukayrÄ«yah','Albukairyah')
df['city'] = df['city'].str.replace('Khamīs','Khamis')
df['city'] = df['city'].str.replace('AlZulfi','Zulfi')
df['city'] = df['city'].str.replace('Abhā','Abha')
df['city'] = df['city'].str.replace('Riyad','Riyadh')
df['city'] = df['city'].str.replace('AlRiyadh','Riyadh')
df['city'] = df['city'].str.replace('â€˜Unayzah','Unayzah')
df['city'] = df['city'].str.replace('‘Unayzah','Unayzah')
df['city'] = df['city'].str.replace('Saudi','Remote')
df['city'] = df['city'].str.replace('AlHufÅ«f','Alhufuf')
df['city'] = df['city'].str.replace('AlKhubar«f','AlKhobar')
df['city'] = df['city'].str.replace('AlKhubar','AlKhobar')
df['city'] = df['city'].str.replace('Al-Bahah','AlBahah')
df['city'] = df['city'].str.replace('AlMajmaâ€˜ah','AlMajardah')
df['city'] = df['city'].str.replace('AlMajmaâ€˜ah','AlMajardah')
df['city'] = df['city'].str.replace('Riyadhh','Riyadh')
df['city'] = df['city'].str.replace('Ḩafar','HafarAlbatin')
df['city'] = df['city'].str.replace('Bedaah','Madinah')
df['city'] = df['city'].str.replace('Qal‘at','Qatif')
df['city'] = df['city'].str.replace('AdDawādimī','AlDuwadimi')
df['city'] = df['city'].str.replace('AlYasmeen','Riyadh')
df['city'] = df['city'].str.replace('Beesh','Bisha')
df['city'] = df['city'].str.replace('Ḩuraymila','Huraymila')
df['city'] = df['city'].str.replace('AlBakhraa','Remote')
df['city'] = df['city'].str.replace('AlHufūf','AlHufuf')
df['city'] = df['city'].str.replace('First','Remote')
df['city'] = df['city'].str.replace('Khamis','Khamis_Mushait')
df['city'] = df['city'].str.replace('AlJazirah','Remote')
df['city'] = df['city'].str.replace('Dasimah','Riyadh')
df['city'] = df['city'].str.replace('Chalon','Remote')
df['city'] = df['city'].str.replace('AlMalga','Riyadh')
df['city'] = df['city'].str.replace('AlMalaz','Riyadh')
df['city'] = df['city'].str.replace('Tuwaiq','Riyadh')
df['city'] = df['city'].str.replace('AlSulaimaniyah','Riyadh')
df['city'] = df['city'].str.replace('Sakākah','Sakakah')
df['city'] = df['city'].str.replace('Jawharat','Riyadh')
df['city'] = df['city'].str.replace('AlBadai','Albadai')
df['city'] = df['city'].str.replace('AlGhamas','AlNamas')
df['city'] = df['city'].str.replace('AlHamra','Riyadh')
df['city'] = df['city'].str.replace('AlUlayah','Riyadh')
df['city'] = df['city'].str.replace('AlWaha','Riyadh')
df['city'] = df['city'].str.replace('AlRoudah','Jeddah')
df['city'] = df['city'].str.replace('Azzarqa','AlBahah')
df['city'] = df['city'].str.replace('AlRaid','Riyadh')
df['city'] = df['city'].str.replace('AlRahmaniyah','Riyadh')
df['city'] = df['city'].str.replace('AlWorood','Riyadh')
df['city'] = df['city'].str.replace('Baha','AlBahah')
df['city'] = df['city'].str.replace('AlNuzha','Riyadh')
df['city'] = df['city'].str.replace('AlQadesiyah','Riyadh')
df['city'] = df['city'].str.replace('Ţurayf','Turaif')
df['city'] = df['city'].str.replace('AlBukayrīyah','Albukairyah')
df['city'] = df['city'].str.replace('Sulaimaniyah','Riyadh')
df['city'] = df['city'].str.replace('Ishbayliyah','Riyadh')
df['city'] = df['city'].str.replace('Yanbu‘','Yanbu')
df['city'] = df['city'].str.replace('Radhwa','Remote')
df['city'] = df['city'].str.replace('King','Jeddah')
df['city'] = df['city'].str.replace('Alsnaeia','Madinah')
df['city'] = df['city'].str.replace('Eastern','Alsharqiyah')
df['city'] = df['city'].str.replace('Hailil','Hail')
df['city'] = df['city'].str.replace('AlMajal','Riyadh')
df['city'] = df['city'].str.replace('AlAlBahahh','AlBahah')
df['city'] = df['city'].str.replace('Ḑubā','Duba')
df['city'] = df['city'].str.replace('Ashifa','Taif')
df['city'] = df['city'].str.replace('Ḩā’il','Hail')
df['city'] = df['city'].str.replace('AlMajma‘ah','AlMajmaah')
df['city'] = df['city'].str.replace('AlLīth','AlLith')
df['city'] = df['city'].str.replace('Ras','RasTanura')
df['city'] = df['city'].str.replace('AlSaadah','Riyadh')
df['city'] = df['city'].str.replace('Quwayiyah','AlQuwayiyah')
df['city'] = df['city'].str.replace('Ain','AlAin')
df['city'] = df['city'].str.replace('Western','Remote')
df['city'] = df['city'].str.replace('Alhawiyah','Taif')
df['city'] = df['city'].str.replace('AlQaţīf','Qatif')
df['city'] = df['city'].str.replace('AlTaawon','Riyadh')
df['city'] = df['city'].str.replace('Alfaysalyah','Riyadh')
df['city'] = df['city'].str.replace('Dubai','Remote')
df['city'] = df['city'].str.replace('Fiji','Madinah')
df['city'] = df['city'].str.replace('AlNakheel','Riyadh')
df['city'] = df['city'].str.replace('Sihat','Saihat')
df['city'] = df['city'].str.replace('AlJawf','Sakakah')
df['city'] = df['city'].str.replace('Bahra','Jeddah')
df['city'] = df['city'].str.replace('AlKhalidiyah','Jeddah')
df['city'] = df['city'].str.replace('India','Remote')
df['city'] = df['city'].str.replace('Mhayel','MahayelAseer')
df['city'] = df['city'].str.replace('Alhufuf','AlHufuf')
df['city'] = df['city'].str.replace('Second','Remote')
df['city'] = df['city'].str.replace('Pulaski','Remote')
df['city'] = df['city'].str.replace('Ash','Alsharqiyah')
df['city'] = df['city'].str.replace('Star','Zulfi')
df['city'] = df['city'].str.replace('United','Remote')
df['city'] = df['city'].str.replace('Al‘Ulá','Al-Ula')
df['city'] = df['city'].str.replace('Milwaukee','Remote')
df['city'] = df['city'].str.replace('Sarat','SaratUbaida')
df['city'] = df['city'].str.replace('‘Remote‘ar','Remote')
df['city'] = df['city'].str.replace('Rhyd','Remote')
df['city'] = df['city'].str.replace('Domat','DumatAlJandal')
df['city'] = df['city'].str.replace('AlMutawakkel','Dammam')
df['city'] = df['city'].str.replace('Tanajib','Remote')
df['city'] = df['city'].str.replace('á¸¨uraymila','Huraymila')
df['city'] = df['city'].str.replace('AlDeri','Jeddah')
df['city'] = df['city'].str.replace('Finland','Remote')
df['city'] = df['city'].str.replace('Indiana','Remote')
df['city'] = df['city'].str.replace('AlHailda','Taif')
df['city'] = df['city'].str.replace('Umm','Umm_al-Hamam')
df['city'] = df['city'].str.replace('Shaqrā','Shaqra')
df['city'] = df['city'].str.replace('AlRemotetawiya','Remote')
df['city'] = df['city'].str.replace('Remoteir','Remote')
df['city'] = df['city'].str.replace('99999','Remote')
df['city'] = df['city'].str.replace('South','Remote')
df['city'] = df['city'].str.replace('Barbourville','Remote')
df['city'] = df['city'].str.replace('Stockton','Remote')
df['city'] = df['city'].str.replace('New','Remote')
df['city'] = df['city'].str.replace('Chesterfield','Remote')
df['city'] = df['city'].str.replace('Silver','Jeddah')
df['city'] = df['city'].str.replace('Ta\'if,','Taif')
df['city'] = df['city'].str.replace('Florida','Remote')
df['city'] = df['city'].str.replace('Remotenapolis','Remote')
df['city'] = df['city'].str.replace('AlAlAin','AlAin')
df['city'] = df['city'].str.replace('‘Ar‘ar','Arar')
df['city'] = df['city'].str.replace('‘Afīf','Afif')
df['city'] = df['city'].str.replace('Paterson','Remote')
df['city'] = df['city'].str.replace('AlDoha','Remote')
df['city'] = df['city'].str.replace('Najrān','Najran')
df['city'] = df['city'].str.replace('Northern','Remote')
df['city'] = df['city'].str.replace('Buffalo','Remote')
df['city'] = df['city'].str.replace('Tarboro','Remote')
df['city'] = df['city'].str.replace('Å¢urayf','Turaif')
df['city'] = df['city'].str.replace('Banban','Riyadh')
df['city'] = df['city'].str.replace('AlOlaya','Riyadh')
df['city'] = df['city'].str.replace('Alehsaa','Ahsa')
df['city'] = df['city'].str.replace('AlHada','Taif')
df['city'] = df['city'].str.replace('East','Alsharqiyah')
df['city'] = df['city'].str.replace('Khurais','Riyadh')
df['city'] = df['city'].str.replace('AlBakhrā’','Remote')
df['city'] = df['city'].str.replace('Laurel','Makkah')
df['city'] = df['city'].str.replace('Oakland','Remote')
df['city'] = df['city'].str.replace('Oak','Remote')
df['city'] = df['city'].str.replace('Alcoa','Remote')
df['city'] = df['city'].str.replace('Oberlin','Remote')
df['city'] = df['city'].str.replace('Shaqra’','Shaqra')
df['city'] = df['city'].str.replace('AlḨa’ir','AlHair')
df['city'] = df['city'].str.replace('AlHezam','Makkah')
df['city'] = df['city'].str.replace('Lake','Dhahban')
df['city'] = df['city'].str.replace('Pasadena','Remote')
df['city'] = df['city'].str.replace('AlJabal','Jeddah')
df['city'] = df['city'].str.replace('Hasah','Ahsa')
df['city'] = df['city'].str.replace('Dawmat','DumatAlJandal')
df['city'] = df['city'].str.replace('AlBahr','Yanbu')
df['city'] = df['city'].str.replace('Haradh','Ahsa')
df['city'] = df['city'].str.replace('Remotena','Remote')
df['city'] = df['city'].str.replace('AlMualimeen','AlHufuf')
df['city'] = df['city'].str.replace('AlAlBahah','AlBahah')
df['city'] = df['city'].str.replace('AlAziziah','Jeddah')
df['city'] = df['city'].str.replace('Kansas','Remote')
df['city'] = df['city'].str.replace('Wellington','Remote')
df['city'] = df['city'].str.replace('Hamilton','Remote')
df['city'] = df['city'].str.replace('Harrisonburg','Remote')
df['city'] = df['city'].str.replace('Sumter','Remote')
df['city'] = df['city'].str.replace('Souderton','Remote')
df['city'] = df['city'].str.replace('AlArtawiya','Remote')
df['city'] = df['city'].str.replace('AlKhafji','Khafji')
df['city'] = df['city'].str.replace('Sakaka','Sakakah')
df['city'] = df['city'].str.replace('AlMidhnab','AlMuthneb')
df['city'] = df['city'].str.replace('AlQurayyat','Qurayyat')
df['city'] = df['city'].str.replace('AlQurayat','Qurayyat')
df['city'] = df['city'].str.replace('turaif','Turaif')
df['city'] = df['city'].str.replace('AlQasim','AlQassim')
df['city'] = df['city'].str.replace('Qaseem','AlQassim')

In [17]:
print(df['city'].unique())

['Riyadh' 'Jeddah' 'Remote' 'Dammam' 'Khobar' 'Rabigh' 'Makkah' nan
 'Yanbu' 'Dhahran' 'Madinah' 'Jubail' 'Jazan' 'Ahsa' 'Tabuk' 'AlQassim'
 'Taif' 'Najran' 'Hail' 'AlBahah' 'HafarAlbatin' 'Alsharqiyah' 'AlKharj'
 'AlKhobar' 'Qatif' 'AlDuwadimi' '' 'Bisha' 'Huraymila' 'AlHufuf'
 'AlMuthneb' 'Khamis_Mushait' 'Abha' 'AlDhahran' 'Sakakahh' 'Albadai'
 'AlNamas' 'Buraydah' 'Yadma' 'Turaif' 'Badr' 'Qurayyat' 'Albukairyah'
 'AlAflaj' 'Hinakiyah' 'Duba' 'Unayzah' 'AlMajmaah' 'Khafji' 'AlLith'
 'RasTanura' 'AlShuqaiq' 'AlQuwayiyah' 'AlAin' 'Sharorah' 'Abqaiq'
 'Dhahban' 'Saihat' 'Balqarn' 'Thuwal' 'MahayelAseer' 'Zulfi' 'AlMajardah'
 'AlUla' 'SaratUbaida' 'Arar' 'DumatAlJandal' 'AlWajh' 'LeMéridien'
 'Umm_al-Hamam' 'Harad' 'AlHair' 'RiyadhBin' 'AlMubarraz' 'AlMajma'
 'Rafha' 'Neom' 'Baysh' 'OmAlsharqiyahr' 'AlFarah' 'Sakakah' 'Safwa'
 'Afif' 'Asir' 'AlDiriyah' 'Shaqra' 'AlKhurmah']


In [18]:
#convert blank space to null values
df['city']=df['city'].replace(r'', np.NaN)

In [19]:
#fill null values
df['city']=df['city'].fillna('NotDefined')

In [20]:
for char in spec_chars:
    df['city'] = df['city'].str.replace(char, '')

In [21]:
print(df['city'].unique())

['Riyadh' 'Jeddah' 'Remote' 'Dammam' 'Khobar' 'Rabigh' 'Makkah'
 'NotDefined' 'Yanbu' 'Dhahran' 'Madinah' 'Jubail' 'Jazan' 'Ahsa' 'Tabuk'
 'AlQassim' 'Taif' 'Najran' 'Hail' 'AlBahah' 'HafarAlbatin' 'Alsharqiyah'
 'AlKharj' 'AlKhobar' 'Qatif' 'AlDuwadimi' 'Bisha' 'Huraymila' 'AlHufuf'
 'AlMuthneb' 'KhamisMushait' 'Abha' 'AlDhahran' 'Sakakahh' 'Albadai'
 'AlNamas' 'Buraydah' 'Yadma' 'Turaif' 'Badr' 'Qurayyat' 'Albukairyah'
 'AlAflaj' 'Hinakiyah' 'Duba' 'Unayzah' 'AlMajmaah' 'Khafji' 'AlLith'
 'RasTanura' 'AlShuqaiq' 'AlQuwayiyah' 'AlAin' 'Sharorah' 'Abqaiq'
 'Dhahban' 'Saihat' 'Balqarn' 'Thuwal' 'MahayelAseer' 'Zulfi' 'AlMajardah'
 'AlUla' 'SaratUbaida' 'Arar' 'DumatAlJandal' 'AlWajh' 'LeMéridien'
 'UmmalHamam' 'Harad' 'AlHair' 'RiyadhBin' 'AlMubarraz' 'AlMajma' 'Rafha'
 'Neom' 'Baysh' 'OmAlsharqiyahr' 'AlFarah' 'Sakakah' 'Safwa' 'Afif' 'Asir'
 'AlDiriyah' 'Shaqra' 'AlKhurmah']


In [22]:
#add new column region >> extract first word from city column
df['Regions'] = df['city'].str.split(' ').str[0]

In [23]:
df.loc[df['Regions'].str.contains('Jeddah|Makkah|Rabigh|Taif|Thuwal|Dhahban|AlKhurmah|AlLith'), 
                  'Regions'] = 'Makkah'

df.loc[df['Regions'].str.contains('Riyadh|AlKharj|AlHair|AlMajma|AlDuwadimi|AlQuwayiyah|Shaqra|AlAflaj|Afif|AlDiriyah|Huraymila|Zulfi|UmmalHamam'), 
                  'Regions'] = 'Riyadh'

df.loc[df['Regions'].str.contains('Dammam|AlKhobar|Alsharqiyah|Jubail|Dhahran|Khobar|RasTanura|Abqaiq|Qatif|AlHufuf|Ahsa|Safwa|Khafji|Harad|Umm_al-Hamam|Saihat|AlMubarraz|HafarAlbatin'), 
                  'Regions'] = 'Eastern'

df.loc[df['Regions'].str.contains('Khamis_Mushait|Abha|AlMajardah|MahayelAseer|AlFarah|Balqarn|Bisha|SaratUbaida|AlNamas|KhamisMushait'), 
                  'Regions'] = 'Asir'

df.loc[df['Regions'].str.contains('Jazan|AlShuqaiq|Baysh'), 
                  'Regions'] = 'Jazan'

df.loc[df['Regions'].str.contains('Madinah|Yanbu|Badr|Hinakiyah|Al-Ula|AlUla'), 
                  'Regions'] = 'Medina'

df.loc[df['Regions'].str.contains('Buraydah|AlQassim|AlMuthneb|Albadai|Albukairyah|Unayzah'), 
                  'Regions'] = 'AlQassim'

df.loc[df['Regions'].str.contains('Tabuk|AlWajh|Neom|Duba'), 
                  'Regions'] = 'Tabuk'

df.loc[df['Regions'].str.contains('Hail'), 
                  'Regions'] = 'Hail'

df.loc[df['Regions'].str.contains('Najran|Sharorah|Yadma'), 
                  'Regions'] = 'Najran'

df.loc[df['Regions'].str.contains('Sakakah|Qurayyat|DumatAlJandal'), 
                  'Regions'] = 'AlJawf '

df.loc[df['Regions'].str.contains('AlBahah|AlAin'), 
                  'Regions'] = 'AlBahah'

df.loc[df['Regions'].str.contains('Turaif|Rafha|Arar'), 
                  'Regions'] = 'Northern Borders'

df.loc[df['Regions'].str.contains('NotDefined'), 
                  'Regions'] = 'NotDefind'

df.loc[df['Regions'].str.contains('Remote|LeMéridien'),
                  'Regions'] = 'Remote'


# |As|Ta|Abu|Aţ|Le|Ha|Ar|Om|AlImam|At

In [24]:
pd.options.display.max_rows = 4000
df['Regions'].value_counts()

Riyadh              22735
Remote               9124
Makkah               6522
Eastern              6379
Medina               1845
Jazan                 781
AlQassim              373
Tabuk                 216
Asir                  198
NotDefind              75
Hail                   49
AlBahah                46
AlJawf                 42
Northern Borders       34
Najran                 30
Name: Regions, dtype: int64

### Industries column Cleaning 

In [25]:
for char in spec_chars:
    df['industries'] = df['industries'].str.replace(char, '')

In [26]:
#Replace blank space column to null values

df['industries']=df['industries'].replace(r'', np.NaN)

In [27]:
#drop null values
df=df.dropna(subset=['industries'])



### Processing industries column 

In [28]:
df.industries.sample(60)

7392                                           Oil  Energy
8882     Information Technology and Services Computer S...
31074                   ElectricalElectronic Manufacturing
39151            Automotive Staffing and Recruiting Retail
38705                                    Civil Engineering
25216    TransportationTruckingRailroad Maritime Logist...
39344                                          Hospitality
42006                                           Accounting
23799                                       Consumer Goods
8684                                              Internet
7729       Higher Education Research International Affairs
47599                  Information Technology and Services
7456                                                Retail
45757                           Logistics and Supply Chain
19985    Computer Software Information Technology and S...
10224                            Government Administration
3935                                       Human Resourc

In [29]:
# print how many unique values in industries column

df.industries.nunique()

3451

In [30]:
split1=df.groupby("industries")

In [31]:
split1=list(split1)

In [32]:
len(split1)

3451

In [33]:
print(split1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
#extract from industries groupind data 

df['industry_cat'] = df['industries'].str.extract(
    r'(Administrative|Business Supplies and Equipment|Chemicals|Business Development|Building Materials|Capital Markets|Business Supplies and Equipment|Business Development|Building Materials|Banking|Aviation  Aerospace|Biotechnology|Broadcast Media|Automotive|Arts and Crafts|Architecture  Planning|Apparel  Fashion|Automotive|Animation|AccountingAuditing|Market Research|Oil  Energy|Insurance|Hospitality|Food Production|Food  Beverages|Information Technology and Services|Hospital  Health Care|Fashion|Financial Services|Architecture  Planning|AirlinesAviation|Accounting|Civic  Social Organization|Civil Engineering|Commercial Real Estate|Computer  Network Security|Computer Games|Computer Hardware|Computer Networking|Computer Software|Construction|Consulting|Consumer Electronics|Consumer Goods|Consumer Services|Cosmetics|Customer Service Sales|Dairy|Defense  Space|Design|ELearning|Education|ElectricalElectronic|Entertainment|Environmental Services|Events Services|Executive Services|Facilities Services|Farming|Fine Art|Furniture|Graphic Design|Glass Ceramics|Government Administration|Government Relations|Graphic Design|Human Resources|Health Care Provider|Individual  Family Services|Information Services|Internet|Investment|Law|Legal Service|Leisure Travel  Tourism|Logistic and Supply Chain|Luxury Goods  Jewelry|Machinery|Marketing and Advertising|Market Research|Management Consulting|Media|Medical|Mental Health Care|Military|Mobile Game|Music|Museums|Nanotechnology|Newspapers|Oil  Energy|Online Media|Real Estate|Security|Retail|Sports|Supermarkets|Telecommunication|Writing|Warehousing|Utilites|Venture Capital|Wholesale|Sporting Goods|Research|Publishing|Public Safety|Public Policy|Public Relations|Photography|Packaging and Containers|Quality Assurance|Purchasing Supply Chain|OutsourcingOffshoring|Pharmaceuticals|Plastics|Training|Motion Pictures|International Trade|International Affairs|Import and Export|Industrial Automation|Maritime|Mechanical|Program Development|Project Management|Mining|Philanthropy|Printing|TransportationTruckingRailroad|Restaurants|Health Wellness and Fitness|Paper  Forest Products|Higher Education|Renewables  Environment|Semiconductors|Staffing and Recruiting|Textiles|Tobacco|Translation and Localization|Venture Capital|Veterinary|Alternative Dispute Resolution|Analyst|Aviation  Aerospace|Hospital  Health Care|Industrial Automation )')




In [35]:
print(df['industry_cat'].value_counts())

Information Technology and Services    6554
Construction                           3937
Marketing and Advertising              2036
Consumer Goods                         1658
Computer Software                      1572
ElectricalElectronic                   1491
Food  Beverages                        1427
Hospital  Health Care                  1376
Staffing and Recruiting                1232
Retail                                 1179
Oil  Energy                            1029
Hospitality                            1016
Management Consulting                   893
Civil Engineering                       854
Internet                                842
Mechanical                              834
Consumer Services                       825
Restaurants                             603
Leisure Travel  Tourism                 602
Pharmaceuticals                         594
Chemicals                               593
Information Services                    579
Human Resources                 

In [36]:
#drop null values
df=df.dropna(subset=['industry_cat'])

In [37]:
print(df['industry_cat'].unique())

['Banking' 'Government Administration' 'Consumer Goods'
 'Aviation  Aerospace' 'Accounting' 'Information Technology and Services'
 'Construction' 'Financial Services' 'Management Consulting' 'Mechanical'
 'Marketing and Advertising' 'Automotive' 'Internet' 'Oil  Energy'
 'Design' 'Hospital  Health Care' 'Consumer Electronics'
 'Architecture  Planning' 'Food Production' 'Facilities Services'
 'Restaurants' 'Food  Beverages' 'Investment' 'Pharmaceuticals'
 'Entertainment' 'Higher Education' 'TransportationTruckingRailroad'
 'Online Media' 'Real Estate' 'Leisure Travel  Tourism' 'Retail'
 'Civil Engineering' 'ElectricalElectronic' 'Computer Software'
 'Import and Export' 'Arts and Crafts' 'Medical' 'Apparel  Fashion'
 'Staffing and Recruiting' 'Building Materials' 'Defense  Space'
 'Commercial Real Estate' 'Information Services' 'OutsourcingOffshoring'
 'Environmental Services' 'Hospitality' 'Health Wellness and Fitness'
 'AirlinesAviation' 'Human Resources' 'Maritime' 'Market Research'
 

In [38]:
#Replace blank space column to null values
df['industry_cat']=df['industry_cat'].replace(r'', np.NaN)

In [39]:
df.isnull().sum()

linkedin_id         6
position_id         0
position            0
company             0
location            0
level              17
date                0
job_functions       0
industries          0
description      1449
day                 0
month               0
quarter             0
city                0
Regions             0
industry_cat        0
dtype: int64

### Jop function column Cleaning  

In [40]:
for char in spec_chars:
    df['job_functions'] = df['job_functions'].str.replace(char, '')

In [41]:
#Replace blank space column to null values
df['job_functions']=df['job_functions'].replace(r'', np.NaN)

In [42]:
#drop null values
df=df.dropna(subset=['job_functions'])


### Level column Cleaning 

In [43]:
for char in spec_chars:
    df['level'] = df['level'].str.replace(char, '')

In [44]:
df['level']=df['level'].fillna('Not Applicable')

### Company column Cleaning 

In [45]:
for char in spec_chars:
    df['company'] = df['company'].str.replace(char, '')

In [46]:
df['company']=df['company'].fillna('other')

### position column Cleaning 

In [47]:
for char in spec_chars:
    df['position'] = df['position'].str.replace(char, '')

### Create a new column Total Positions in each company

In [48]:
df['TotalPositions'] = df['company'].str.split(' ').str[0]

In [49]:
df['TotalPositions'] = df.groupby('company')['position'].transform('count')

### Create a new column Positions in each company By Month

In [50]:
df['PositionsByMonth'] = df.groupby('month')['position'].transform('count')

In [51]:
MonthPositions=np.unique(df[['month','PositionsByMonth']], axis=0)

In [52]:
df_MonthPositions = pd.DataFrame(MonthPositions)

In [53]:
print(df_MonthPositions)

     0     1
0    1  2339
1    2  2531
2    3  2160
3    4  2209
4    5  2389
5    6  2724
6    7  2571
7    8  2879
8    9  4031
9   10  4860
10  11  6071
11  12  7809


In [54]:
CompanyPositions = df.groupby('company')['TotalPositions'].sum()

In [55]:
df['TotalPositions'] = df.groupby('company')['position'].transform('count')

In [56]:
CompanyPositions = df.groupby('company').agg(TotalPositions = ('TotalPositions', 'sum'))

print(CompanyPositions)

                                        TotalPositions
company                                               
                                                     9
  Financial Services                                 1
  Non woven products industry factory                9
 Abdulrahman AlOtaishan  Sons Group CO               1
 Alamjaad Group                                      1
...                                                ...
نجمة الصحراء                                         4
هدايا جَوَى                                          1
وزارة الثقافة Ministry of Culture                    1
وكالة بيت البرند للدعاية والاعلان                    1
وهج الماس                                            1

[5604 rows x 1 columns]


In [57]:
pd.options.display.max_rows = 4000
CompanyPositions.value_counts()

TotalPositions
1                 2438
4                  941
9                  475
16                 347
25                 232
36                 153
49                 112
64                  94
81                  76
121                 58
100                 55
144                 51
169                 42
289                 35
196                 31
256                 29
225                 27
324                 25
361                 22
441                 19
400                 17
676                 15
529                 15
484                 13
1296                10
961                 10
625                  9
1089                 9
784                  9
576                  9
841                  8
1225                 7
1156                 7
1024                 7
900                  7
2601                 6
2025                 6
1764                 6
1849                 5
3481                 5
2500                 5
2116                 5
4096               

### Drop Uneeded columns

In [58]:
#drop linkedin_id and position_id
df= df.drop(['linkedin_id'], axis=1)
df= df.drop(['position_id'], axis=1)
df= df.drop(['location'], axis=1)
df= df.drop(['date'], axis=1)


### Processing Description column 

In [59]:
#Convert data type to string 
df['description'] = df['description'].astype('str')

In [60]:
for char in spec_chars:
    df['description'] = df['description'].str.replace(char, ' ')

In [61]:
df= df.dropna(subset=['description'])

In [62]:
# Checking the uniqu values for all categorical features's values
for col in df:
    if df[col].dtype=="object":
        print(col,":",df[col].unique())

position : ['Head of Branding Governance Unit ' 'Employee Engagement Specialist'
 'Account Manager GPS  Riyadh' ... 'Consultant PICU'
 'اخصائي عناية مركزة قلب أطفال' 'Group Exercise  Personal Training']
company : ['Riyad Bank' 'PepsiCo' 'EY' ... 'Smartmessage' 'Diversephds'
 'Atyab AlKaram International']
level : ['MidSenior level' 'Not Applicable' 'Director' 'Associate' 'Entry level'
 'Internship' 'Executive']
job_functions : ['Marketing Management' 'Human Resources' 'Sales Business Development' ...
 'Project Management Human Resources Information Technology'
 'Sales and Business Development' 'Management and Manufacturing']
industries : ['Banking' 'Consumer Goods Food  Beverages'
 'Accounting Financial Services' ...
 'Hospital  Health Care Human Resources'
 'Food  Beverages Food Production Real Estate'
 'Manufacturing Financial Services and Retail']
description : ['nan'
 'Auto req ID  199735BRJob DescriptionPepsiCo Saudi Snacks For Foods Limited established in 1995 with 2 plants in Ri

In [63]:
df.description.unique()

array(['nan',
       'Auto req ID  199735BRJob DescriptionPepsiCo Saudi Snacks For Foods Limited established in 1995 with 2 plants in Riyadh   Dammam   more than 2 500 employees in KSA   UAE  Our brands include Lays  Doritos  Sunbites  Cheetos  Tasali   Quaker Our employees are at the heart of PepsiCo  Through the Company s commitment to Talent Sustainability  we continue to support the development of employees  At PepsiCo  you get the best of both worlds  an entrepreneurs mindset plus reach and resources  Our collaborative culture and worldwide presence generate a stream of new opportunities to define the future and propel your lifes work  Bring your unique perspective  Bring curiosity  Bring ingenuity  and drive  Well give you a platform to be daring on a global scale Job PurposeTo maintain the internal Employee Engagement in the organization and support in material and content design and communication Principal AccountabilitiesOwn the engagement calendar and draft a plan with respec

In [64]:
def remove_unwanted_characters(s):
    s = re.sub('\*', ' ', str(s))
    s = re.sub('/', ' ', str(s))
    s = re.sub('_', ' ', str(s))
    s = re.sub('\?', ' ', str(s))
    s = re.sub('%', ' ', str(s))
    s = re.sub('@', ' ', str(s))
    s = re.sub('#', ' ', str(s))
    s = re.sub('!', ' ', str(s))
    s = re.sub(',', ' ',str(s))
    s = re.sub('\+', ' ', str(s))
    s = re.sub('&', ' ', str(s))
    #s = re.sub('\d', ' ', str(s)) delete digit NO
    return s


In [65]:
#Remove common text in job describtion

def rem_fluff(data):
    data = data.replace('please share resume', " ")
    data = data.replace('call me', " ")
    data = data.replace('if you are a', " ")
    data = data.replace('if you are an', " ")
    data = data.replace('job description', " ")
    data = data.replace('if you are', " ")
    data = data.replace('reply', " ")
    data = data.replace("please apply today applicants must be authorized to work in the u.s.please apply directly to by clicking 'click here to apply' with your word resume looking forward to receiving your resume and going over the position in more detail with you.- not a fit for this position click the link at the bottom of this email to search all of our open positions.looking forward to receiving your resume cybercoderscybercoders inc is proud to be an equal opportunity employerall qualified applicants will receive consideration for employment without regard to race color religion sex national origin disability protected veteran status or any other characteristic protected by law.your right to work - in compliance with federal law all persons hired will be required to verify identity and eligibility to work in the united states and to complete the required employment eligibility verification document form upon hire.copyright  - . cybercoders inc. all rights reserved.", ' ')
    return data

In [66]:
# function to get rid of repeated skills after a '-'
def split_skills(data):
        x = data
        nospace = x.replace(' ', '') # removing spaces
        if '-' in nospace:
            first_half = nospace.split('-')[0] #first half of string before '-'
            second_half = nospace.split('-')[1] # second half of string after '-'
            if first_half == second_half:
                x = list(x.split('-'))[0].strip() #getting only first half of string to return
                return x
            else:
                return x      
        else:
            return x

In [67]:
def drop_corrupt_rows(df):
    df['word_count'] = df['description'].map(lambda x: len(x.split()))
    
    mean_word_count = df['word_count'].mean()
    print('Mean word count per post:', round(mean_word_count))
    
    std_word_count = df['word_count'].std()
    print('std word count per post:', round(std_word_count))
    
    too_short = mean_word_count - 1.5*std_word_count
    too_short = 20
    
    # Drop posts with 20 word or less
    print(len(df), 'before dropping')
    mask = df['word_count'] > too_short
    df = df[mask]
    print(len(df), 'after dropping')

    df.drop(columns='word_count', inplace=True)
    
    return df.reset_index(drop=True)

In [68]:
import re

In [69]:
# Remove unwanted characters and numbers
df['description'] = df['description'].map(remove_unwanted_characters)
df['industries'] = df['industries'].map(remove_unwanted_characters)

In [70]:
# Correct multiple spaces (including \t and \n)
df['description'] = df['description'].map(lambda x: re.sub(r'[^a-zA-Z0-9]',' ', x))
df['industries'] = df['industries'].map(lambda x: re.sub(r'[^a-zA-Z0-9]',' ', x))

In [71]:
# Drop corrupted rows
df = drop_corrupt_rows(df)

Mean word count per post: 339
std word count per post: 271
42573 before dropping
40500 after dropping


In [72]:
# Remove common phrases
df['description'] = df['description'].map(rem_fluff)
df['industries'] = df['industries'].map(rem_fluff)

In [73]:
df['description'] = df['description'].str.lower()

In [74]:
#Number of unique entries were found for each columns.
df.apply(pd.Series.nunique)

position            17856
company              5244
level                   7
job_functions        2407
industries           3151
description         29099
day                    31
month                  12
quarter                 4
city                   74
Regions                15
industry_cat          130
TotalPositions        127
PositionsByMonth       12
dtype: int64

### Extracting Year Of Experience From Job Description

In [75]:
df[['description']]= df[['description']].dropna()


def years_of_ex(description):
    
    if description != np.nan:
        description = description.lower() 
        match = re.search('[^0-9]\+ years|[^0-9]\+ year|[0-9] [0-9]\+ years|[0-9] [0-9]\+ year|\+[0-9]|[0-9] to [0-9] year|[0-9] to [0-9] years|[0-9] to [0-9] yrs|[0-9] [0-9] yrs|[^0-9] +year|[0-9] +years|..\+ yrs|[0-9] [0-9] +years|entry|junior', description) 

        if match != None:
            return description[match.start(): match.end()]
    
    return np.nan
            

df['years_of_ex'] = df['description'].apply(years_of_ex)
print(df['years_of_ex'].isnull().sum())

12043


In [76]:
df.loc[24657,'description']

'the quality manager shall have a bachelor degree in the major required with an experience of 12 years in this field and shall have quality certificates and for saudi national  an experience of 6 years in the construction field and shall have quality certificates please apply hereclosing date  31 dec 2020show moreshow less'

In [77]:
df.years_of_ex.unique()

array(['e year', '7  years', '5  years', '5 years', '2 years', nan,
       '8   years', '2  years', 'm year', '5 7 years', '4 years',
       '1 2 years', '3 4 years', '5 6 years', '2 3 years', '0  years',
       'i year', '6 years', '0 years', '8  years', 'junior', '0 2 years',
       '4 6 years', 'a year', '3 years', '1 3 years', '8 years',
       '4  years', '9 years', 'd year', 'f year', 'o year', 'n year',
       '  year', '7 years', '3  years', '3 5 years', 'entry', '2 5 years',
       '5 8 years', 'l year', '2 4 years', '7 8 years', 'r year',
       '4 5 years', '4 to 6 year', '6  years', 's year', 'x year',
       '3 to 4 year', 't year', '0 3 years', '5 to 8 year', '1 years',
       '3 8 years', 'd   year', '6 to 9 year', '2 to 5 year', '6 8 years',
       'g year', '1 to 3 year', '1 5 years', '2 3  years', 'y  year',
       'p  year', 'h year', 'e   year', '6 9 years', '0   years',
       'y year', '0 to 1 year', '1  years', '2 to 4 year', '3 6 years',
       '5 to 6 year', '4

In [78]:
# Extract numbers
df['year_of_ex']=df['years_of_ex'].str.extract('(\d)')

In [79]:
#change data type to calculate the mean 
df['year_of_ex'] = df['year_of_ex'].astype(float)

In [80]:
#calculate the mean of expirence according to level grouping
years=df.groupby('level').agg(years_of_ex = ('year_of_ex', 'mean'))

In [81]:
years

,years_of_ex
level,
Associate,3.166414
Director,3.007446
Entry level,3.402283
Executive,2.920771
Internship,1.867925
MidSenior level,3.563933
Not Applicable,3.233962


In [82]:
#fill null values with mean
df['year_of_ex']=df['year_of_ex'].fillna(df.groupby('level')['year_of_ex'].transform('mean'))

In [83]:
#convert years_experience from float to int 
df['year_of_ex'] = df['year_of_ex'].astype(int)

In [84]:
df.year_of_ex.unique()

array([3, 7, 5, 2, 8, 4, 1, 0, 6, 9])

In [85]:
df.year_of_ex.value_counts()

3    20967
5     5709
0     4264
2     4150
8     1383
4     1369
1     1050
7      880
6      653
9       75
Name: year_of_ex, dtype: int64

In [86]:
df.isnull().sum()

position                0
company                 0
level                   0
job_functions           0
industries              0
description             0
day                     0
month                   0
quarter                 0
city                    0
Regions                 0
industry_cat            0
TotalPositions          0
PositionsByMonth        0
years_of_ex         12043
year_of_ex              0
dtype: int64

In [87]:
df.head(50)

,position,company,level,job_functions,industries,description,day,month,quarter,city,Regions,industry_cat,TotalPositions,PositionsByMonth,years_of_ex,year_of_ex
0,Employee Engagement Specialist,PepsiCo,MidSenior level,Human Resources,Consumer Goods Food Beverages,auto req id 199735brjob descriptionpepsico sa...,6,1,1,Riyadh,Riyadh,Consumer Goods,219,2339,e year,3
1,Account Manager GPS Riyadh,EY,Not Applicable,Sales Business Development,Accounting Financial Services,in a business where are our people are our pro...,6,1,1,Riyadh,Riyadh,Accounting,101,2339,7 years,7
2,Business Analyst,تطوير تقنيات التعليم Tatweer Educational Tech...,MidSenior level,Business Development Sales,Information Technology and Services,the ideal candidate will design organize and...,6,1,1,Riyadh,Riyadh,Information Technology and Services,16,2339,5 years,5
3,Talent Acquisition Advisor,WSP in the Middle East,Associate,Finance,Construction,think bigger scale think higher profile thin...,6,1,1,Riyadh,Riyadh,Construction,43,2339,5 years,5
4,Organisational Design Specialist,Aventus Global Talent,MidSenior level,Human Resources Design,Financial Services Banking,organisational design specialist eastern regi...,6,1,1,Remote,Remote,Financial Services,28,2339,5 years,5
5,Quality Specialist,Takamol Holding,Associate,Analyst Consulting Quality Assurance,Management Consulting,main accountabilities familiarity with kpis...,6,1,1,Riyadh,Riyadh,Management Consulting,23,2339,2 years,2
6,Events Operations Manager,Aggreko,MidSenior level,Management Manufacturing,Marketing and Advertising Mechanical or Indust...,about aggreko plcaround the world people bus...,6,1,1,Jeddah,Makkah,Marketing and Advertising,24,2339,e year,3
7,Business Analyst,ALKHALEEJ COMPUTERS ELECTRONIC SYSTEMS,Associate,Business Development Sales,Information Technology and Services Banking,the senior business analyst will be a key cont...,6,1,1,Riyadh,Riyadh,Information Technology and Services,6,2339,NaN,3
8,Program ManagerEasternSouthernNorthernWestern ...,Elm Company,Associate,Project Management Information Technology,Information Technology and Services,role purposeresponsible for planning implemen...,6,1,1,Riyadh,Riyadh,Information Technology and Services,22,2339,8 years,8
9,Project manager,AECOM,MidSenior level,Project Management,Architecture Planning Civil Engineering Const...,saudi arabia ar riyad riyadh job summary ma...,6,1,1,Riyadh,Riyadh,Architecture Planning,364,2339,2 years,2


In [88]:
#remove stopword from describtion
spec_char=["in","or","and","from","is","a","that","with","at","of"]
for chars in spec_char:
    
    df['description'] = df['description'].str.replace(chars, '')






In [89]:
#extract degees word from description
df['degree']=df['description'].str.extract('(\w+ degree \w+ |\w+  degrees \w+ )')

In [90]:
#replace degree 
df['degree'] = df['degree'].str.replace('degree','')


In [91]:
df['degree']=df['degree'].str.replace('bachelors|bachelor|bachelor|university|bachelors|qualificationsbachelors|bs|requirementsbachelors|qualificationsbachelor|experiencebachelor|experiencebachelors|timebachelors|ba', 'bachelor')



In [92]:
df['degree'].isnull().sum()

37706

In [93]:
df.degree.value_counts()

bachelor  cludg                               111
mster  fered                                   85
hve  qulifiction                               49
technicl  preferred                            45
msters  preferred                              45
ssocite  diplom                                43
bachelor  tht                                  42
bchels  holder                                 42
to  level                                      42
bchels  required                               39
relted  required                               38
s  required                                    36
requirements  qulifiction                      35
bchels  requirement                            31
msters  will                                   26
idelly  educted                                26
s  preferred                                   24
be  qulified                                   24
bchel  level                                   21
msters  preferble                              20


### Extracting Degree From Job Description

In [94]:
def degree(description):
    
    if description != np.nan:
        description = description.lower() 
        #match = re.search('\+ diploma degree|\+ master|\+phd|\+ bacholer|\+ masters|\+ bacholers|\+msc|\+ms', description) 
        match = re.search(r'bachelor|master|masters| ms | bs | phd | msc |university|technicl degree|mster|bchel|bchels|msters|diplom| hve |needbchels|experiencebchels|requirementsbchels|college|qulifictionsbchels|electricl |undergrdute|grdute|mechnicl', description)      

        #match = re.search('\w+ degree', description) 



        if match != None:
            return description[match.start(): match.end()]

    return np.nan
            

df['degree'] = df['description'].apply(degree)
print(df['degree'].isnull().sum())

8128


In [95]:
df['degree'] = df['degree'].str.replace(' ','')

In [96]:
df.degree.value_counts()

hve                   14016
bchel                  6896
bs                     2993
mster                  1386
diplom                 1223
grdute                 1039
ms                     1018
electricl               895
university              867
mechnicl                861
college                 338
qulifictionsbchels      269
requirementsbchels      164
undergrdute             121
experiencebchels         88
msc                      86
technicldegree           45
phd                      45
needbchels               22
Name: degree, dtype: int64

In [97]:

df['degree'] = df['degree'].str.replace('undergrdute','student')



In [98]:
#bachelor

df['degree'] = df['degree'].str.replace('needbchels','bachelor')
df['degree'] = df['degree'].str.replace('experiencebchels','bachelor')
df['degree'] = df['degree'].str.replace('requirementsbchels','bachelor')
df['degree'] = df['degree'].str.replace('qulifictionsbchels','bachelor')


In [99]:
#bachelor

bachelor1=["bchel","bs","grdute","college","university"]

for chars12 in bachelor1:
    
    df['degree'] = df['degree'].str.replace(chars12,'bachelor')

In [100]:

print(df['degree'].isnull().sum())

8128


In [101]:
#master

master1=["master","msc","masters","ms","mster","msters","masterter"," master"]

for chars1 in master1:
    
    df['degree'] = df['degree'].str.replace(chars1,'master')

In [102]:
#diploma

diploma1=["technicldegree","electricl","mechnicl"]

for chars11 in diploma1:
    
    df['degree'] = df['degree'].str.replace(chars11,'diplom')

In [103]:
#phd
df['degree'] = df['degree'].str.replace('phd','Doctorate') 

In [104]:

#hve

df['degree'] = df['degree'].str.replace('hve','Higher Vocational Education') 

In [105]:
#change diplom to diploma
df['degree'] = df['degree'].str.replace('diplom','diploma') 

In [106]:
df.degree.value_counts()

Higher Vocational Education    14016
bachelor                       12676
diploma                         3024
master                          2490
student                          121
Doctorate                         45
Name: degree, dtype: int64

In [123]:
df['degree_int']=df['degree'].str.extract('(Higher Vocational Education|bachelor|diploma|master|student|Doctorate)')

In [124]:
# add new column extracting values from degree and replace it with numnbers to calculate the mean and fill null values with it with 
df['degree_int'] = df['degree_int'].str.replace('Higher Vocational Education','1') 
df['degree_int'] = df['degree_int'].str.replace('bachelor','2') 
df['degree_int'] = df['degree_int'].str.replace('diploma','3') 
df['degree_int'] = df['degree_int'].str.replace('master','4') 
df['degree_int'] = df['degree_int'].str.replace('student','5') 
df['degree_int'] = df['degree_int'].str.replace('Doctorate','6') 


In [125]:
df.degree_int.value_counts()

1    14016
2    12676
3     3024
4     2490
5      121
6       45
Name: degree_int, dtype: int64

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40500 entries, 0 to 40499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   position          40500 non-null  object
 1   company           40500 non-null  object
 2   level             40500 non-null  object
 3   job_functions     40500 non-null  object
 4   industries        40500 non-null  object
 5   description       40500 non-null  object
 6   day               40500 non-null  int64 
 7   month             40500 non-null  int64 
 8   quarter           40500 non-null  int64 
 9   city              40500 non-null  object
 10  Regions           40500 non-null  object
 11  industry_cat      40500 non-null  object
 12  TotalPositions    40500 non-null  int64 
 13  PositionsByMonth  40500 non-null  int64 
 14  years_of_ex       28457 non-null  object
 15  year_of_ex        40500 non-null  int64 
 16  degree            32372 non-null  object
 17  degree_int  

In [127]:
df['degree_int']=df['degree_int'].replace(r'', np.NaN)

In [128]:
#change data type to float to correctly calculate the mean
df['degree_int'] = df['degree_int'].astype(float)

In [129]:
#convert degree to integer values

df['degree_int']=df['degree_int'].fillna(df.groupby('level')['degree_int'].transform('mean'))



In [130]:
#convert float to int 
df['degree_int'] = df['degree_int'].astype(int)

In [131]:
#convert float to int 
df['degree_int'] = df['degree_int'].astype(str)

In [132]:
#rereplace the numbers to string
df['degree_int'] = df['degree_int'].replace('1','Higher Vocational Education') 
df['degree_int'] = df['degree_int'].replace('2','bachelor') 
df['degree_int'] = df['degree_int'].replace('3','diploma') 
df['degree_int'] = df['degree_int'].replace('4','master') 
df['degree_int'] = df['degree_int'].replace('5','student') 
df['degree_int'] = df['degree_int'].replace('6','Doctorate')

In [133]:
df.isnull().sum()

position                0
company                 0
level                   0
job_functions           0
industries              0
description             0
day                     0
month                   0
quarter                 0
city                    0
Regions                 0
industry_cat            0
TotalPositions          0
PositionsByMonth        0
years_of_ex         12043
year_of_ex              0
degree               8128
degree_int              0
degrees                 0
dtype: int64

In [134]:
#fill new column with values of degree_int column
df['degrees'] = df['degree_int']

In [135]:
df.degrees.value_counts()

Higher Vocational Education    22144
bachelor                       12676
diploma                         3024
master                          2490
student                          121
Doctorate                         45
Name: degrees, dtype: int64

In [136]:
df.sample(50)

,position,company,level,job_functions,industries,description,day,month,quarter,city,Regions,industry_cat,TotalPositions,PositionsByMonth,years_of_ex,year_of_ex,degree,degree_int,degrees
7108,Legal Advisor,Dr Mohammed AlMuhanna Partners,MidSenior level,Legal Consulting,Legal Services,requirements grdution ledg university b m...,21,4,2,Riyadh,Riyadh,Legal Service,2,2209,5 years,5,bachelor,bachelor,bachelor
11658,Learning Development Manager Jeddah,Saudi German Hospitals,MidSenior level,Health Care Provider Administrative Human Reso...,Hospital Health Care Health Wellness and Fitn...,job brief we re seekg lerng development mng...,16,6,2,Jeddah,Makkah,Hospital Health Care,19,2724,NaN,3,bachelor,bachelor,bachelor
35133,Accountant,Confidential,Entry level,AccountingAuditing Finance Engineering,Construction Machinery Accounting,job summry implement the ccountg system ensur...,10,12,4,Riyadh,Riyadh,Construction,724,7809,3 years,3,bachelor,bachelor,bachelor
31280,Store Associate,Common Ksa,Entry level,Sales Business Development,Retail,position ste ssociteoverviewth position resp...,25,11,4,Dammam,Eastern,Retail,8,6071,NaN,3,bachelor,bachelor,bachelor
13865,Human Resources Specialist,iHomeآي هوم,Entry level,Human Resources,Furniture Consumer Goods Retail,key responsibilities personnel speciltopertion...,12,7,3,Makkah,Makkah,Furniture,14,2571,3 to 5 year,3,bachelor,bachelor,bachelor
25392,Infrastructure Planning Analyst0006 5000817HO,stc,Entry level,Information Technology,Information Technology and Services Telecommun...,jobs t th level re responsible f supptg mtg ...,22,10,4,Riyadh,Riyadh,Information Technology and Services,368,4860,NaN,3,NaN,Higher Vocational Education,Higher Vocational Education
14457,Sales Manager MOH,BD,Entry level,Sales Business Development,Medical Devices,sles mnger mohjob description summryto mnge th...,20,7,3,Remote,Remote,Medical,13,2571,3 5 years,3,bachelor,bachelor,bachelor
3898,Sales Account Manager,Converged Generation Communication Co,MidSenior level,Sales Business Development,Information Technology and Services Telecommun...,the sles ccount mnger will be reptg to the sec...,7,3,1,Riyadh,Riyadh,Information Technology and Services,3,2160,2 years,2,bachelor,bachelor,bachelor
38953,Director Corporate Communications,Confidential,Executive,Marketing Public Relations WritingEditing,Oil Energy,develop mnge implement communiction plns f c...,24,12,4,Alsharqiyah,Eastern,Oil Energy,724,7809,d year,2,bachelor,bachelor,bachelor
9728,HSE Manager,Air Products,Not Applicable,Management Manufacturing,Chemicals,position summryjob description qulifictionsbs...,24,5,2,Remote,Remote,Chemicals,290,2389,5 years,5,master,master,master


In [137]:
#drop unneeded columns 
df= df.drop(['degree'], axis=1)
df= df.drop(['degree_int'], axis=1)
df= df.drop(['years_of_ex'], axis=1)

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40500 entries, 0 to 40499
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   position          40500 non-null  object
 1   company           40500 non-null  object
 2   level             40500 non-null  object
 3   job_functions     40500 non-null  object
 4   industries        40500 non-null  object
 5   description       40500 non-null  object
 6   day               40500 non-null  int64 
 7   month             40500 non-null  int64 
 8   quarter           40500 non-null  int64 
 9   city              40500 non-null  object
 10  Regions           40500 non-null  object
 11  industry_cat      40500 non-null  object
 12  TotalPositions    40500 non-null  int64 
 13  PositionsByMonth  40500 non-null  int64 
 14  year_of_ex        40500 non-null  int64 
 15  degrees           40500 non-null  object
dtypes: int64(6), object(10)
memory usage: 4.9+ MB


In [139]:
df.head(50)

,position,company,level,job_functions,industries,description,day,month,quarter,city,Regions,industry_cat,TotalPositions,PositionsByMonth,year_of_ex,degrees
0,Employee Engagement Specialist,PepsiCo,MidSenior level,Human Resources,Consumer Goods Food Beverages,uto req id 199735brjob descriptionpepsico sud...,6,1,1,Riyadh,Riyadh,Consumer Goods,219,2339,3,bachelor
1,Account Manager GPS Riyadh,EY,Not Applicable,Sales Business Development,Accounting Financial Services,busess where re our people re our products ...,6,1,1,Riyadh,Riyadh,Accounting,101,2339,7,Higher Vocational Education
2,Business Analyst,تطوير تقنيات التعليم Tatweer Educational Tech...,MidSenior level,Business Development Sales,Information Technology and Services,the idel cidte will design gnize modify the...,6,1,1,Riyadh,Riyadh,Information Technology and Services,16,2339,5,bachelor
3,Talent Acquisition Advisor,WSP in the Middle East,Associate,Finance,Construction,thk bigger scle thk higher prile thk ground ...,6,1,1,Riyadh,Riyadh,Construction,43,2339,5,Higher Vocational Education
4,Organisational Design Specialist,Aventus Global Talent,MidSenior level,Human Resources Design,Financial Services Banking,gntionl design specilt estern region sudi rb...,6,1,1,Remote,Remote,Financial Services,28,2339,5,bachelor
5,Quality Specialist,Takamol Holding,Associate,Analyst Consulting Quality Assurance,Management Consulting,m ccountbilities fmilirity kp buildg rev...,6,1,1,Riyadh,Riyadh,Management Consulting,23,2339,2,bachelor
6,Events Operations Manager,Aggreko,MidSenior level,Management Manufacturing,Marketing and Advertising Mechanical or Indust...,bout ggreko plcround the wld people busesses...,6,1,1,Jeddah,Makkah,Marketing and Advertising,24,2339,3,bachelor
7,Business Analyst,ALKHALEEJ COMPUTERS ELECTRONIC SYSTEMS,Associate,Business Development Sales,Information Technology and Services Banking,the seni busess nlyst will be key contct use...,6,1,1,Riyadh,Riyadh,Information Technology and Services,6,2339,3,Higher Vocational Education
8,Program ManagerEasternSouthernNorthernWestern ...,Elm Company,Associate,Project Management Information Technology,Information Technology and Services,role purposeresponsible f plnng implementtion...,6,1,1,Riyadh,Riyadh,Information Technology and Services,22,2339,8,Higher Vocational Education
9,Project manager,AECOM,MidSenior level,Project Management,Architecture Planning Civil Engineering Const...,sudi rbi r riyd riydh job summry mnges one ...,6,1,1,Riyadh,Riyadh,Architecture Planning,364,2339,2,bachelor


### >> Extract csv file

In [140]:
#extract csv file after preprocessing

df.to_csv('data_jobs.csv')